In [1]:
import os
import glob
import pandas as pd

import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool
from bokeh.models import Range1d
from scipy import stats
import pandas as pd
import math

from bokeh.layouts import row

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
data_path = 'C:/Users/lorenzo/Desktop/20201022'
working_path = "C:/Users/lorenzo/Desktop/git/PhD_codes/Mechanosensation/Python_code/Motor_localization_analysis/"

os.chdir(working_path)
new_dir = "Data_PaQa_levels\\"
working_data_path = working_path + new_dir
if not os.path.exists(new_dir):
    os.mkdir(new_dir)
    
os.chdir(data_path)
extension = 'csv'
list_csv = []
for root, dirs, files in os.walk(data_path, topdown=False):
    for name in files:
        if extension in name:
            list_csv.append(os.path.join(root, name))

for file in list_csv:
    df_temp = pd.read_csv(file, sep=',', na_values='*')
    split_name=file.rsplit('Data_',2)[1].rsplit('_',100)
    #print(file)
    #print(split_name)
    file_name = file.rsplit("\\",2)[2]
    df_temp['Strain'] = split_name[0]
    if 'sol' in file:
        df_temp['Growth']='Solid'
    else:
        df_temp['Growth']='Liquid'
    df_temp['Bio_Rep']=int(split_name[len(split_name)-1].rsplit('BR=', 2)[1].rsplit('.',2)[0])
    df_temp.to_csv(working_data_path + 'enhanced_' + file_name, index = None, header=True)
    list_enhanced_csv = []

    
for root, dirs, files in os.walk(working_data_path, topdown=False):
    for name in files:
        if extension in name:
            list_enhanced_csv.append(os.path.join(root, name))

combined_csv = pd.concat([pd.read_csv(f) for f in list_enhanced_csv ], sort=False)
os.chdir(working_path)
combined_csv.to_csv( "SummaryPaQa-Copy1.csv", index=False, encoding='utf-8-sig')

In [37]:
def plottingAllData(df, means, labels, indexes, param, colors):
    df.head()
    p = bokeh.plotting.figure(
        width=600, 
        height=600, 
        y_range=labels, 
        x_axis_type='linear',
        x_axis_label = 'Fluorescence intensity',
        title="Motor "+param+" on solid vs liquid environement"
    )
    for i, index in enumerate(indexes):
        p.circle(
            source=df.loc[index, :],
            x=param, 
            y=jitter('Labels', width=0.3, range=p.y_range),
            color = colors[i],
            alpha=0.3,
            #legend = labelsAll[i]
        )
    p.circle(
        source = means,
        x = param,
        y = 'Labels',
        size = 10,
        line_color = 'black',
        fill_color = 'white',
        legend = "medians"
    )
    p.add_tools(HoverTool(
            tooltips=[
                ('Strain', '@{Strain}'),
                ('Growth condition', '@{Growth}'),
                (param, '@{'+param+'}'),
                ('Biological replicate', '@{Bio_Rep}')
            ],
    ))
    return p

def plottingData(df, means, wt, order, strain_to_remove, growth, param, colors, scale, GraphTitle):
    df2 = df.copy()
    means2 = means.copy()
    
    if isinstance(growth, list):
        for strain in strain_to_remove:
            df2 = df2.drop(df2[(df2['Strain'] == strain)].index)
            means2 = means2.drop(means2[(means2['Strain'] == strain)].index)
        for strain in order:
            df3 = df2.copy()
            means3 = means2.copy()
        val = 0.5
        labels = [None]*(2+len(order)*2)
        indexes = [None]*(2+len(order)*2)
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), 'Labels'] = val
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), 'Labels'] = val
        labels[0] = wt+' '+'Liquid'
        indexes[0] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid'))
        df3.loc[((df2['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        labels[1] = wt+' '+'Solid'
        indexes[1] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid'))
        val = val + 2
        n=2
        for o, ordering in enumerate(order):
            for a, g in enumerate(growth):
                df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(g)), 'Labels'] = val
                means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(g)), 'Labels'] = val
                labels[n] = ordering + ' ' +g
                indexes[n] = ((df3['Strain'] == (ordering)) & df3['Growth'].str.match(g))
                val = val + 1
                n=n+1
        lookupdf2=df3.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
        p = bokeh.plotting.figure(
            width=600, 
            height=600, 
            x_range=labels, 
            y_axis_type=scale,
            y_axis_label = 'Fluorescence intensity',
            title=GraphTitle+" "+param+" on "+growth[0]+" vs. "+growth[1]+" in "+scale+" scale"
        )
        p.xaxis.major_label_orientation = math.pi/4
        p.xaxis.major_label_text_font_size = "9pt"
        for i, index in enumerate(indexes):
            p.circle(
                source=df3.loc[index, :],
                x=jitter('Labels', width=0.3, range=p.y_range),
                y=param, 
                color = colors[i],
                alpha=0.3,
                #legend = labelsAll[i]
            )
        p.circle(
            source = means3,
            x = 'Labels',
            y = param,
            size = 10,
            line_color = 'black',
            fill_color = 'white',
            legend_label = "medians"
        )
        p.add_tools(HoverTool(
                tooltips=[
                    ('Strain', '@{Strain}'),
                    ('Growth condition', '@{Growth}'),
                    (param, '@{'+param+'}'),
                    ('Biological replicate', '@{Bio_Rep}')
                ],
        ))
    else:
        print('Not a list')
        if growth == 'Liquid':
            ungrowth = 'Solid'
        else:
            ungrowth = 'Liquid'
        for strain in strain_to_remove:
            df2 = df2.drop(df2[(df2['Strain'] == strain)].index)
            means2 = means2.drop(means2[(means2['Strain'] == strain)].index)
        for strain in order:
            df3 = df2.drop(df2[((df2['Strain'] == strain) & (df2['Growth'] == ungrowth))].index)
            means2 = means2.drop(means2[((means2['Strain'] == strain) & (means2['Growth'] == ungrowth))].index)
        means3=means2
        val = 0.5
        labels = [None]*(2+len(order))
        indexes = [None]*(2+len(order))
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), 'Labels'] = val
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), 'Labels'] = val
        WT_mean=np.array(means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), param])
        Norm_mean=WT_mean/np.mean(WT_mean)
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), param]=Norm_mean
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), param] = df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), param]/np.mean(WT_mean)
        labels[0] = wt+' '+'Liquid'
        indexes[0] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid'))
        print(wt+' Liquid mean='+str(WT_mean))
        df3.loc[((df2['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        Strain_mean=np.array(means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), param])
        Norm_mean=Strain_mean/WT_mean
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), param]=Norm_mean
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), param] = df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), param]/np.mean(WT_mean)
        labels[1] = wt+' '+'Solid'
        indexes[1] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid'))
        print(wt+' Solid mean='+str(Strain_mean))
        print(wt+' Solid fold increase='+str(Strain_mean/np.mean(WT_mean)))
        val = val + 2
        for o, ordering in enumerate(order):
            df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), 'Labels'] = val
            means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), 'Labels'] = val
            Strain_mean=np.array(means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), param])
            Norm_mean=Strain_mean/np.mean(WT_mean)
            means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), param]=Norm_mean
            print(ordering+' mean='+str(Strain_mean))
            print(ordering+' fold increase='+str(Strain_mean/np.mean(WT_mean)))
            df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), param] = df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), param]/np.mean(WT_mean)
            labels[o+2] = ordering+' '+growth
            indexes[o+2] = ((df3['Strain'] == (ordering)) & df3['Growth'].str.match(growth))
            val = val + 1
        lookupdf2=df3.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
        p = bokeh.plotting.figure(
            width=600, 
            height=600, 
            x_range=labels, 
            y_axis_type=scale,
            y_axis_label = 'Fold increase',
            title=GraphTitle+" "+param+" on "+growth
        )
        p.xaxis.major_label_orientation = math.pi/4
        p.xaxis.major_label_text_font_size = "9pt"
        for i, index in enumerate(indexes):
            p.circle(
                source=df3.loc[index, :],
                x=jitter('Labels', width=0.3, range=p.y_range),
                y=param, 
                color = colors[i],
                alpha=0.3,
                #legend = labelsAll[i]
            )
        p.circle(
            source = means3,
            x = 'Labels',
            y = param,
            size = 10,
            line_color = 'black',
            fill_color = 'white',
            #legend_label = "medians"
        )
        p.add_tools(HoverTool(
                tooltips=[
                    ('Strain', '@{Strain}'),
                    ('Growth condition', '@{Growth}'),
                    (param, '@{'+param+'}'),
                    ('Biological replicate', '@{Bio_Rep}')
                ],
        ))
    return p, means3

def plottingDataUnique(df, means, wt, order, strain_to_remove, growth, param, colors, scale, GraphTitle):
    df2 = df.copy()
    means2 = means.copy()
    
    if isinstance(growth, list):
        for strain in strain_to_remove:
            df2 = df2.drop(df2[(df2['Strain'] == strain)].index)
            means2 = means2.drop(means2[(means2['Strain'] == strain)].index)
        for strain in order:
            df3 = df2.copy()
            means3 = means2.copy()
        val = 0.5
        labels = [None]*(2+len(order)*2)
        indexes = [None]*(2+len(order)*2)
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), 'Labels'] = val
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), 'Labels'] = val
        labels[0] = wt+' '+'Liquid'
        indexes[0] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid'))
        df3.loc[((df2['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        labels[1] = wt+' '+'Solid'
        indexes[1] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid'))
        val = val + 2
        n=2
        for o, ordering in enumerate(order):
            for a, g in enumerate(growth):
                df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(g)), 'Labels'] = val
                means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(g)), 'Labels'] = val
                labels[n] = ordering + ' ' +g
                indexes[n] = ((df3['Strain'] == (ordering)) & df3['Growth'].str.match(g))
                val = val + 1
                n=n+1
        lookupdf2=df3.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
        p = bokeh.plotting.figure(
            width=600, 
            height=600, 
            x_range=labels, 
            y_axis_type=scale,
            y_axis_label = 'Fluorescence intensity',
            title=GraphTitle+" "+param+" on "+growth[0]+" vs. "+growth[1]+" in "+scale+" scale"
        )
        p.xaxis.major_label_orientation = math.pi/4
        p.xaxis.major_label_text_font_size = "9pt"
        for i, index in enumerate(indexes):
            p.circle(
                source=df3.loc[index, :],
                x=jitter('Labels', width=0.3, range=p.y_range),
                y=param, 
                color = colors[i],
                alpha=0.3,
                #legend = labelsAll[i]
            )
        p.circle(
            source = means3,
            x = 'Labels',
            y = param,
            size = 10,
            line_color = 'black',
            fill_color = 'white',
            legend_lable = "medians"
        )
        p.add_tools(HoverTool(
                tooltips=[
                    ('Strain', '@{Strain}'),
                    ('Growth condition', '@{Growth}'),
                    (param, '@{'+param+'}'),
                    ('Biological replicate', '@{Bio_Rep}')
                ],
        ))
    else:
        if growth == 'Liquid':
            ungrowth = 'Solid'
        else:
            ungrowth = 'Liquid'
        for strain in strain_to_remove:
            df2 = df2.drop(df2[(df2['Strain'] == strain)].index)
            means2 = means2.drop(means2[(means2['Strain'] == strain)].index)
        for strain in order:
            df3 = df2.drop(df2[((df2['Strain'] == strain) & (df2['Growth'] == ungrowth))].index)
            means3 = means2.drop(means2[((means2['Strain'] == strain) & (means2['Growth'] == ungrowth))].index)
        val = 0.5
        labels = [None]*(1+len(order))
        indexes = [None]*(1+len(order))
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), 'Labels'] = val
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), 'Labels'] = val
        WT_mean=list(means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), param])
        Norm_mean=WT_mean/WT_mean
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), param]=Norm_mean
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), param] = df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), param]/WT_mean
        labels[0] = wt+' '+'Liquid'
        indexes[0] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid'))
        val = val + 1
        for o, ordering in enumerate(order):
            df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), 'Labels'] = val
            means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), 'Labels'] = val
            Strain_mean=float(means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), param])
            Norm_mean=Strain_mean/WT_mean
            means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), param]=Norm_mean
            df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), param] = df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), param]/WT_mean
            labels[o+1] = ordering+' '+growth
            indexes[o+1] = ((df3['Strain'] == (ordering)) & df3['Growth'].str.match(growth))
            val = val + 1
        lookupdf2=df3.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
        p = bokeh.plotting.figure(
            width=600, 
            height=600, 
            x_range=labels, 
            y_axis_type=scale,
            y_axis_label = 'Fluorescence intensity',
            title=GraphTitle+" "+param+" on "+growth
        )
        p.xaxis.major_label_orientation = math.pi/4
        p.xaxis.major_label_text_font_size = "9pt"       
        for i, index in enumerate(indexes):
            print(df3.loc[index, param].mean())
            p.circle(
                source=df3.loc[index, :],
                x=jitter('Labels', width=0.3, range=p.y_range),
                y=param, 
                color = colors[i],
                alpha=0.3,
                #legend = labelsAll[i]
            )
        print(means3)
        p.circle(
            source = means3,
            x = 'Labels',
            y = param,
            size = 10,
            line_color = 'black',
            fill_color = 'white',
            legend_label = "medians"
        )
        p.add_tools(HoverTool(
                tooltips=[
                    ('Strain', '@{Strain}'),
                    ('Growth condition', '@{Growth}'),
                    (param, '@{'+param+'}'),
                    ('Biological replicate', '@{Bio_Rep}')
                ],
        ))
    return p

def getDFStats(df, param):
    Strains=list(df.Strain.unique())
    Growth=list(df.Growth.unique())
    Reps=list(df.Bio_Rep.unique())
    inds = [None]*(len(Strains)*len(Growth))
    labels = [None]*(len(Strains)*len(Growth))
    labelsAll = [None]*(len(Strains)*len(Growth)*len(Reps))
    indexes = [None]*(len(Strains)*len(Growth)*len(Reps))
    n = 0
    m = 0
    for strain in Strains:
        for growth in Growth:
            inds[n] = (df['Strain'] == strain) & (df['Growth'] == growth)
            labels[n] = strain+' '+growth
            n= n + 1
            for rep in Reps:
                indexes[m] = (df['Strain'] == strain) & (df['Growth'] == growth) & (df['Bio_Rep'] == rep)
                labelsAll[m] = strain+' '+growth
                m = m + 1
    df['Labels'] = 0
    #df.insert(2, "Labels", 0, True)
    vertical_position = [None]*len(labels)
    val=0.5        
    for s, strain in enumerate(labels):
        vertical_position[s] = val
        df.loc[inds[s], 'Labels'] = val
        val = val+1        
    means = df.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].agg(['median', 'count']).rename(columns={'median': param, 'count': 'N'}).reset_index()
    #means = df.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
    stds = df.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].std().to_frame().reset_index()
    return means, stds, indexes, labels

In [38]:
df_long = pd.read_csv('SummaryPaQa-Copy1.csv', sep=',', na_values='*')
df_long['PaQa_ratio']=df_long['MeanCellIntensity_PaQa']/df_long['MeanCellIntensity_RFP']
lengths = df_long.groupby(['Strain', 'Growth', 'Bio_Rep'])['CellLength'].median().to_frame().reset_index()
l_max=lengths.CellLength.max()
df=df_long.loc[(df_long['CellLength'] <= l_max)].copy()
df_long.head()

,Frame,CellID,Area,CellLength,CellSinuosity,CellWidth,MaxCellIntensity_RFP,MeanCellIntensity_RFP,MedianCellIntensity_RFP,MinCellIntensity_RFP,MaxCellIntensity_PaQa,MeanCellIntensity_PaQa,MedianCellIntensity_PaQa,MinCellIntensity_PaQa,Strain,Growth,Bio_Rep,PaQa_ratio
0,1,1,0.642200,2.171924,1.012087,0.411096,1276,937.000000,979.5,287,12875,10150.585526,10896.0,2218,cpdA-fliC-,Liquid,1,10.833069
1,1,2,0.714025,2.452315,1.072699,0.473207,1753,1296.479290,1363.0,387,18139,13682.911243,14488.0,3412,cpdA-fliC-,Liquid,1,10.553899
2,1,3,0.794300,2.826543,1.049439,0.495025,1345,991.287234,1028.5,354,15702,11890.271277,12240.0,2825,cpdA-fliC-,Liquid,1,11.994779
3,1,4,0.747825,2.490391,1.067156,0.495025,1598,1122.960452,1181.0,412,12326,9029.966102,9502.0,2181,cpdA-fliC-,Liquid,1,8.041215
4,1,5,0.667550,2.450402,1.052882,0.495025,1699,1338.987342,1360.0,711,18226,13319.455696,14275.5,5389,cpdA-fliC-,Liquid,1,9.947410


In [39]:
#param='MeanCellIntensity_mNeonGreen'
param='PaQa_ratio'
    
[means, stds, indexes, labels] = getDFStats(df_long, param)

#stds = df.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].std().to_frame().reset_index()
#df.to_csv( "DataSet.csv", index=False, encoding='utf-8-sig')
#means_pilT
means.head(100)

,Strain,Growth,Labels,Bio_Rep,PaQa_ratio,N
0,WT,Liquid,38.5,1,0.474211,94
1,WT-Px2,Liquid,36.5,1,0.527023,181
2,WT-Px2,Solid,37.5,1,1.768271,229
3,WT-alex,Liquid,32.5,1,0.573665,125
4,WT-alex,Solid,33.5,1,2.057077,55
5,WT-lolo,Liquid,34.5,1,0.629268,258
6,WT-lolo,Solid,35.5,1,1.394498,363
7,cpdA-fliC-,Liquid,0.5,1,8.891907,346
8,cpdA-fliC-,Liquid,0.5,2,8.487138,595
9,cpdA-fliC-,Liquid,0.5,3,9.106100,937


In [40]:
colors = bokeh.palettes.d3['Category20'][20]*3
growth='Liquid'

p1, me1 = plottingData(df_long, means, 'fliC-lolo', ['pilH-fliC-','cpdA-fliC-', 'pilH-cpdA-fliC-', 'pilG-cpdA-fliC-', 'pilG-fliC-', 'cyaB-fliC-', 'pilG-cyaB-fliC-', 'pilH-cyaB-fliC-', 'pilTU-fliC-'], ['WT-Px2','WT-lolo','WT-alex','fliC-','pilAT8V', 'pilB-fliC-T8V', 'pilT-fliC-T8V', 'pilT-pilA-fliC-', 'pilH-'], growth, param, colors, 'linear', '')
#p2, me2 = plottingData(df_long, means, 'fliC-lolo', ['pilH-fliC-','cpdA-fliC-', 'pilG-fliC-', 'cyaB-fliC-', 'pilTU-fliC-', 'pilG-cyaB-fliC-', 'pilH-cyaB-fliC-', 'pilG-cpdA-fliC-', 'pilH-cpdA-fliC-'], ['WT-Px2','WT-lolo','WT-alex','fliC-','pilAT8V', 'pilB-fliC-T8V', 'pilT-fliC-T8V', 'pilT-pilA-fliC-', 'pilH-'], growth, param, colors, 'log', '')
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None
p1.xaxis.minor_tick_line_color = None
p1.yaxis.minor_tick_line_color = None
p1.y_range=Range1d(0,33)
p1.output_backend = 'svg'
bokeh.io.show(p1)
#bokeh.io.show(row(p1,p2))

Not a list
fliC-lolo Liquid mean=[0.71876818 0.34404673 0.83255467]
fliC-lolo Solid mean=[2.01878754 2.53624806 2.47896103]
fliC-lolo Solid fold increase=[3.19534653 4.01438551 3.92371133]
pilH-fliC- mean=[10.12544536  8.74868479  9.05803649]
pilH-fliC- fold increase=[16.02660312 13.84745993 14.33710328]
cpdA-fliC- mean=[8.89190707 8.48713803 9.10609987]
cpdA-fliC- fold increase=[14.07415285 13.43348247 14.41317822]
pilH-cpdA-fliC- mean=[10.93336413 12.14036353  9.05318959]
pilH-cpdA-fliC- fold increase=[17.30538079 19.21582518 14.32943158]
pilG-cpdA-fliC- mean=[7.3700967  2.57965878 3.04700741 2.97661933 2.73832036]
pilG-cpdA-fliC- fold increase=[11.66542413  4.0830962   4.82281784  4.71140725  4.3342265 ]
pilG-fliC- mean=[0.29296532 0.31665676 0.26532618]
pilG-fliC- fold increase=[0.4637069  0.50120582 0.41995953]
cyaB-fliC- mean=[0.29359093 0.28100342 0.28081422]
cyaB-fliC- fold increase=[0.46469713 0.44477356 0.44447408]
pilG-cyaB-fliC- mean=[0.35696244 0.3246434  0.32256305]
pilG-

In [36]:
p3 = plottingDataUnique(df_long, means, 'fliC-lolo', ['pilH-fliC-','cpdA-fliC-', 'pilG-fliC-', 'cyaB-fliC-', 'pilTU-fliC-', 'pilG-cyaB-fliC-', 'pilH-cyaB-fliC-', 'pilG-cpdA-fliC-', 'pilH-cpdA-fliC-'], ['WT-Px2','WT-lolo','WT-alex','fliC-','pilAT8V', 'pilB-fliC-T8V', 'pilT-fliC-T8V', 'pilT-pilA-fliC-', 'pilH-'], 'Liquid', param, colors, 'linear', '')
bokeh.io.show(p3)

TypeError: unsupported operand type(s) for /: 'list' and 'list'

In [9]:
p4, means3 = plottingData(df_long, means, 'fliC-lolo', ['pilH-fliC-','cpdA-fliC-', 'pilH-cpdA-fliC-', 'pilG-fliC-', 'cyaB-fliC-', 'pilG-cyaB-fliC-', 'pilH-cyaB-fliC-', 'pilG-cpdA-fliC-', 'pilT-pilA-fliC-'], ['pilH-','WT', 'pilAT8V', 'pilB-fliC-T8V', 'pilT-fliC-T8V','WT-Px2','WT-lolo','WT-alex','fliC-'], 'Liquid', param, colors, 'linear', '')
bokeh.io.show(p4)
means3.head(100)

Not a list
<class 'pandas.core.series.Series'>


ValueError: operands could not be broadcast together with shapes (1332,) (3,) 